# Satellite images of dust (split-window difference)

**Libraries needed:**

In [1]:
#---Cloud search libraries
import s3fs #---The installation of this package on conda makes me nervous for the environment stability
import requests
import fnmatch

#---Accessory libraries
import datetime
import xarray as xr
import netCDF4
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as feature

ModuleNotFoundError: No module named 'matplotlib.pyplot'

**Set the datetime to view:**

In [ ]:
year = 2024
month = 8
day = 24
hour = 12

julian_day = datetime.datetime(year, month, day).strftime('%j')
print(julian_day)

datetime_str = str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2)+' '+str(hour).zfill(2)+'Z'
print(datetime_str)

237
2024-08-24 12Z


**Select region for visualization:**

In [4]:
#--- US Southwest
latitude_north = 40.5
latitude_south = 27.5
longitude_west = -123
longitude_east = -100

**Connecting to AWS remote storage:**

In [5]:
fs = s3fs.S3FileSystem(anon=True)

**Search the AWS database:**

Split-window difference = 10.3um - 12.3um

In [9]:
bucket = 'noaa-goes16'
product = 'ABI-L1b-RadF' #---Full disk ABI radiance

band_13 = '13' #--- 10.3 um
band_15 = '15' #--- 12.3 um

#---Add band selection here (C**)

data_path = bucket + '/' + product + '/'  + str(year) + '/' + julian_day + '/' + str(hour).zfill(2)

files = fs.ls(data_path)

files_band_15 = [file for file in files if fnmatch.fnmatch(file.split('/')[-1], 'OR_ABI-L1b-RadF-M6C' + band_15 + '*')]
files_band_13 = [file for file in files if fnmatch.fnmatch(file.split('/')[-1], 'OR_ABI-L1b-RadF-M6C' + band_13 + '*')]

In [12]:
file_15 = files_band_15[0] #---first is the top-of-the-hour
file_13 = files_band_13[0]

print(file_15)

noaa-goes16/ABI-L1b-RadF/2024/237/12/OR_ABI-L1b-RadF-M6C15_G16_s20242371200205_e20242371209519_c20242371209567.nc


In [13]:
resp_15 = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+file_15[12:])
if str(resp_15) != '<Response [200]>':
    print('b15 file not found in AWS servers')

resp_13 = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+file_13[12:])
if str(resp_13) != '<Response [200]>':
    print('b13 file not found in AWS servers')

**Open the satellite data:**

In [15]:
nc_15 = netCDF4.Dataset(file_15, memory = resp_15.content)
ds_15 = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_15))

nc_13 = netCDF4.Dataset(file_13, memory = resp_13.content)
ds_13 = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_13))

**Get the latitudes and longitudes for the satellite image:**

In [16]:
def calc_latlon(ds):
    # The math for this function was taken from 
    # https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm
    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection
    
    x,y = np.meshgrid(x,y)
    
    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat
    
    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2
    
    #--- Added absolute to remove error
    r_s = (-b - np.sqrt(np.absolute(b**2 - 4*a*c)))/(2*a)
    
    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)
    
    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)
    
    ds = ds.assign_coords({
        "lat":(["y","x"],lat),
        "lon":(["y","x"],lon)
    })
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"
    return ds



def get_xy_from_latlon(ds, lats, lons):
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data
    
    x = ds.x.data
    y = ds.y.data
    
    x,y = np.meshgrid(x,y)
    
    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 
    
    return ((min(x), max(x)), (min(y), max(y)))

In [ ]:
ds_lat_lon_1 = calc_latlon(ds_1)
((x1,x2), (y1, y2)) = get_xy_from_latlon(ds_lat_lon_1, (latitude_south, latitude_north), (longitude_west, longitude_east))
region_1 = ds_lat_lon_1.sel(x=slice(x1, x2), y=slice(y2, y1))

ds_lat_lon_2 = calc_latlon(ds_2)
region_2 = ds_lat_lon_2.sel(x=slice(x1, x2), y=slice(y2, y1))


NameError: name 'np' is not defined

**Convert radiance to brightness temperature:**

* Satellite data has the conversion constants in the metadata

In [ ]:
wl_1 = round(region_1.band_wavelength.values[0],1)
wl_2 = round(region_2.band_wavelength.values[0],1)

Tb_1 = (region_1.planck_fk2/(np.log((region_1.planck_fk1/region_1.Rad)+1)) - region_1.planck_bc1)/region_1.planck_bc2
Tb_2 = (region_2.planck_fk2/(np.log((region_2.planck_fk1/region_2.Rad)+1)) - region_2.planck_bc1)/region_2.planck_bc2

**Create the strings for the save file:**

In [ ]:
date_str = str(datetime_str).replace('-', '_').replace(' ', '_')

**Plot the brightness-temperature difference:**

In [ ]:

BTD = Tb_1 - Tb_2

projection=ccrs.PlateCarree(central_longitude=0)

fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})

#---Custom colorbar
from matplotlib.colors import LinearSegmentedColormap
colors = [(0, '#A9A9A9'), (0.5, 'white'), (1, '#1167b1')]  # +3 = blueish teal, 0 = white, -3 = grey
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)

#---Optional: Set large negative to another color to try to identify overriding cirrus
#cmap.set_under('#DABC94')

#levels = np.linspace(-3, 3, 31)

c=ax.contourf(BTD.lon, BTD.lat, BTD, cmap=cmap, extend='both')#, levels=levels)


clb = plt.colorbar(c, shrink=0.4, pad=0.02, ax=ax)
clb.ax.tick_params(labelsize=15)
clb.set_label('(K)', fontsize=15)

#--- Remove for full-scan
ax.set_extent([longitude_west, longitude_east, latitude_south, latitude_north], crs=ccrs.PlateCarree())
#ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())

ax.set_title("GOES BTD ("+ str(wl_1) +" μm - " + str(wl_2) +" μm) \n("+datetime_str+")", fontsize=20, pad=10)
ax.coastlines(resolution='50m', color='black', linewidth=1)
#ax.add_feature(feature.LAND, zorder=100, edgecolor='#000', facecolor='#000')

fig.set_dpi(200)
fig.savefig("georgesbank_animation/satellite_btd_"+date_str, dpi=200, bbox_inches='tight')
fig.show()